In [2]:
import pandas as pd
import numpy as np

In [3]:
with open('df_topic_and_loc.pkl', 'rb') as pickle_file:
  df_topic_and_loc = pd.compat.pickle_compat.load(pickle_file)

with open('df_bias.pkl', 'rb') as pickle_file:
  df_bias = pd.compat.pickle_compat.load(pickle_file)

In [4]:
df_corpus = df_topic_and_loc.merge(df_bias, on='article_url')
df_corpus

,date_published,publisher,title,text,article_url,topic,topic_list,location,main_site,bias_rating,generalisation,prominence,negative_behaviour,misrepresentation,headline_or_imagery
0,2019-05-23,Dailymail,Iran says it will not surrender even if it is ...,"LONDON, May 23 (Reuters) - Iranian President H...",https://www.dailymail.co.uk/wires/reuters/arti...,Crimes and Arrests | Politics,"[Crimes and Arrests, Politics]","Iran, Islamic Republic of",www.dailymail.co.uk,-1,0,0,1,0,0
1,2019-06-13,Theguardian,MEPs create biggest far-right group in Europea...,"Geert Wilders, left, Matteo Salvini, centre, a...",https://www.theguardian.com/world/2019/jun/13/...,Politics,[Politics],Italy,www.theguardian.com,-1,0,0,1,0,0
2,2019-04-30,Thejc,US army veteran arrested after planning terror...,A US army veteran has been arrested in souther...,https://www.thejc.com/news/us-news/us-army-vet...,Crimes and Arrests | Terrorism and Extremism |...,"[Crimes and Arrests, Terrorism and Extremism, ...",New Caledonia,www.thejc.com,0,0,0,0,0,0
3,2019-07-06,Express,Johnson calls on immigrants to learn English t...,The Tory leadership favourite made the remarks...,https://www.express.co.uk/news/politics/115003...,Politics | Hate Speech and Discrimination,"[Politics, Hate Speech and Discrimination]",United Kingdom,www.express.co.uk,-1,0,0,1,0,0
4,2019-02-01,Christiantoday,"In packed churches and secret masses, papal vi...",In Dubai's overflowing churches and Riyadh's s...,https://www.christiantoday.com/article/in-pack...,Religion,[Religion],Saudi Arabia,www.christiantoday.com,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2019-09-12,Dailymail,Australia presses Iran to release 3 citizens f...,"CANBERRA, Australia (AP) - Australia's foreign...",https://www.dailymail.co.uk/wires/ap/article-7...,Crimes and Arrests | Politics | Minorities and...,"[Crimes and Arrests, Politics, Minorities and ...","Iran, Islamic Republic of",www.dailymail.co.uk,0,0,0,0,0,0
496,2019-03-02,Dailymail,Bernie Sanders gets personal and calls Trump '...,branded\nas 'the most dangerous President in m...,https://www.dailymail.co.uk/news/article-67644...,Business and Economy | Terrorism and Extremism...,"[Business and Economy, Terrorism and Extremism...",Poland,www.dailymail.co.uk,0,0,0,0,0,0
497,2019-08-19,Bbc,Kashmir schools re-open but students stay home,Some schools re-opened in Indian-administered ...,https://www.bbc.co.uk/news/world-asia-india-49...,Politics | Education | Minorities and Human Ri...,"[Politics, Education, Minorities and Human Rig...",India,www.bbc.co.uk,0,0,0,0,0,0
498,2019-07-25,Dailymail,"After 10 years of Boko Haram violence, Nigeria...","MAIDUGURI, Nigeria (AP) - Suicide bombings, ma...",https://www.dailymail.co.uk/wires/ap/article-7...,Business and Economy | Terrorism and Extremism,"[Business and Economy, Terrorism and Extremism]",Nigeria,www.dailymail.co.uk,1,0,0,1,1,0


In [6]:
def show_counts_c1(df_corpus, publisher, c1):
    # Filter publisher
    df_publisher = df_corpus[df_corpus['publisher']==publisher]
    # List possible C1 options that can use group by
    simple_c1 = ['location', 'bias_rating']

    if c1 in simple_c1:
        # Group by c1
        df_count = df_publisher.groupby(c1).size().reset_index(name='count')
        return df_count
    
    elif c1 == 'bias_categories':
        # Prepare expected bias categories
        expected_categories = {
            'generalisation',
            'prominence',
            'negative_behaviour',
            'misrepresentation',
            'headline_or_imagery'
        }
        # Find bias categories that are present in dataframe
        actual_categories = list(set(df_publisher.columns).intersection(expected_categories))
        df_count = df_publisher[actual_categories].sum().rename_axis(c1).reset_index(name='count')
        return df_count
    
    elif c1 == 'topic':
        # Prepare exploded topics
        s_topics = df_publisher[c1].apply(lambda x: x.split(" | "))
        df_count = s_topics.explode().reset_index().groupby(c1).size().reset_index(name='count')
        df_count = df_count.replace('', 'Unknown')
        return df_count
    
    else:
        raise ValueError("Unknown category")

In [25]:
df_publisher.explode('topic_list')

,date_published,publisher,title,text,article_url,topic,topic_list,location,main_site,bias_rating,generalisation,prominence,negative_behaviour,misrepresentation,headline_or_imagery
0,2019-05-23,Dailymail,Iran says it will not surrender even if it is ...,"LONDON, May 23 (Reuters) - Iranian President H...",https://www.dailymail.co.uk/wires/reuters/arti...,Crimes and Arrests | Politics,Crimes and Arrests,"Iran, Islamic Republic of",www.dailymail.co.uk,-1,0,0,1,0,0
0,2019-05-23,Dailymail,Iran says it will not surrender even if it is ...,"LONDON, May 23 (Reuters) - Iranian President H...",https://www.dailymail.co.uk/wires/reuters/arti...,Crimes and Arrests | Politics,Politics,"Iran, Islamic Republic of",www.dailymail.co.uk,-1,0,0,1,0,0
6,2019-08-14,Dailymail,26 candidates to run in Tunisia's early presid...,"TUNIS, Tunisia (AP) - Tunisia's independent el...",https://www.dailymail.co.uk/wires/ap/article-7...,Business and Economy | Politics | Hate Speech ...,Business and Economy,Tunisia,www.dailymail.co.uk,-1,0,0,1,0,0
6,2019-08-14,Dailymail,26 candidates to run in Tunisia's early presid...,"TUNIS, Tunisia (AP) - Tunisia's independent el...",https://www.dailymail.co.uk/wires/ap/article-7...,Business and Economy | Politics | Hate Speech ...,Politics,Tunisia,www.dailymail.co.uk,-1,0,0,1,0,0
6,2019-08-14,Dailymail,26 candidates to run in Tunisia's early presid...,"TUNIS, Tunisia (AP) - Tunisia's independent el...",https://www.dailymail.co.uk/wires/ap/article-7...,Business and Economy | Politics | Hate Speech ...,Hate Speech and Discrimination,Tunisia,www.dailymail.co.uk,-1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,2019-07-25,Dailymail,"After 10 years of Boko Haram violence, Nigeria...","MAIDUGURI, Nigeria (AP) - Suicide bombings, ma...",https://www.dailymail.co.uk/wires/ap/article-7...,Business and Economy | Terrorism and Extremism,Terrorism and Extremism,Nigeria,www.dailymail.co.uk,1,0,0,1,1,0
499,2019-03-04,Dailymail,10 Things to Know for Today - 04 March 2019,"Your daily look at late-breaking news, upcomin...",https://www.dailymail.co.uk/wires/ap/article-6...,Business and Economy | Accidents and Natural D...,Business and Economy,China,www.dailymail.co.uk,-1,0,0,1,0,0
499,2019-03-04,Dailymail,10 Things to Know for Today - 04 March 2019,"Your daily look at late-breaking news, upcomin...",https://www.dailymail.co.uk/wires/ap/article-6...,Business and Economy | Accidents and Natural D...,Accidents and Natural Disasters,China,www.dailymail.co.uk,-1,0,0,1,0,0
499,2019-03-04,Dailymail,10 Things to Know for Today - 04 March 2019,"Your daily look at late-breaking news, upcomin...",https://www.dailymail.co.uk/wires/ap/article-6...,Business and Economy | Accidents and Natural D...,Politics,China,www.dailymail.co.uk,-1,0,0,1,0,0


In [23]:
df_publisher = df_corpus[df_corpus['publisher']=="Dailymail"]
(
    df_publisher
    .groupby(['location', 'bias_rating']).size().reset_index(name='count')
)


,location,bias_rating,count
0,Afghanistan,-1,6
1,Afghanistan,0,3
2,Afghanistan,1,1
3,Argentina,0,1
4,Argentina,1,1
...,...,...,...
97,United Kingdom,0,2
98,United States,-1,6
99,United States,0,3
100,Yemen,-1,3


In [39]:
df_publisher = df_corpus[df_corpus['publisher']=="Dailymail"]

(
    df_publisher
    .melt(
        id_vars='location',
        value_vars=['generalisation', 'prominence', 'negative_behaviour', 'misrepresentation','headline_or_imagery'],
        var_name="bias_categories",
        value_name="count"
        )
    
    .groupby(['location', 'bias_categories']).sum().reset_index()
)

,location,bias_categories,count
0,Afghanistan,generalisation,1
1,Afghanistan,headline_or_imagery,0
2,Afghanistan,misrepresentation,1
3,Afghanistan,negative_behaviour,7
4,Afghanistan,prominence,0
...,...,...,...
295,Yemen,generalisation,0
296,Yemen,headline_or_imagery,0
297,Yemen,misrepresentation,0
298,Yemen,negative_behaviour,3


In [40]:
df_publisher['topic'] = df_publisher['topic'].apply(lambda x: x.split(" | "))
(
    df_publisher
    .explode('topic')
    .groupby(['location', 'topic']).size().reset_index(name="count")
    .replace('', 'Unknown')
)

C:\Users\Amos\AppData\Local\Temp\ipykernel_33864\1783163090.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_publisher['topic'] = df_publisher['topic'].apply(lambda x: x.split(" | "))


,location,topic,count
0,Afghanistan,Business and Economy,4
1,Afghanistan,Crimes and Arrests,1
2,Afghanistan,Education,1
3,Afghanistan,Immigration,1
4,Afghanistan,Minorities and Human Rights,6
...,...,...,...
292,Yemen,Business and Economy,5
293,Yemen,Minorities and Human Rights,3
294,Yemen,Politics,3
295,Yemen,Religion,1


In [32]:
show_counts_c1(df_corpus, "Dailymail", "topic")

,topic,count
0,Unknown,3
1,Accidents and Natural Disasters,14
2,Business and Economy,101
3,Children and Women's Rights,56
4,Crimes and Arrests,39
5,Education,12
6,Hate Speech and Discrimination,29
7,Health,6
8,Immigration,50
9,Minorities and Human Rights,85


In [94]:
## location
# df_count = df_corpus[df_corpus['publisher']=="Dailymail"].groupby("location").size().reset_index(name='counts')
# df_count['pct'] = df_count['counts'] / len(df_corpus[df_corpus['publisher']=="Dailymail"])
# df_count

df_corpus[df_corpus['publisher']=="Dailymail"].groupby("location").size().index.name

'location'